# Decision Trees in Practice

In this notebook we use 6 toy datasets (3 for regression and 3 for classification) to understand how Decision Trees work and get an understanding of their hyperparameters.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import DecisionBoundaryDisplay
# don't get unneccessary warnings
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# You do not need to understand what happens in these functions,
# just execute the cell so you can use the functions below

n_train_reg = 100
n_train_clf = 300

def plot_regression(X, y, model=None):
    # plot a regression dataset (and model predictions)
    plt.figure()
    plt.scatter(X[:, 0], y, s=10, c='#3090C7', alpha=0.7, label='data samples')
    if model is not None:
        X_plot = np.linspace(np.min(X), np.max(X), 1000)
        plt.plot(X_plot, model.predict(X_plot[:, np.newaxis]), '#15317E', linewidth=1., alpha=0.9, label='prediction')
        plt.legend()
    plt.xlabel('x (feature)')
    plt.ylabel('y (target)')
    plt.title('Regression Problem')
    
def plot_classification(X, Y, model=None):
    # plot a classification dataset (and model predictions)
    plt.figure()
    if model is not None:
        DecisionBoundaryDisplay.from_estimator(
            model, X, cmap=plt.cm.RdBu, alpha=0.8, eps=0.5, xlabel="feature 1", ylabel="feature 2", ax=plt.gca()
        )
    # plot the training points
    plt.scatter(X[:, 0], X[:, 1], s=20, c=Y, cmap=ListedColormap(['#FF0000', '#0000FF']), alpha=0.5, label="data samples")
    plt.title("Classification Problem")
    plt.colorbar()
    
def get_linear_regression():
    # generate noisy linear regression dataset
    np.random.seed(15)
    X = np.random.rand(n_train_reg, 1)
    y = -2.5 + 5*X
    y += np.random.randn(n_train_reg, 1) * 0.4
    return StandardScaler(with_std=False).fit_transform(X), y.flatten()

def get_linear_outlier():
    # generate linear regression dataset with outliers
    np.random.seed(15)
    X = np.random.rand(n_train_reg, 1)
    y = -2.5 + 5*X
    y += np.random.randn(n_train_reg, 1) * 0.05
    y[(X>0.7) & (X<0.73)] = 10
    return StandardScaler(with_std=False).fit_transform(X), y.flatten()

def get_nonlinear_regression():
    # generate noisy non-linear regression dataset
    np.random.seed(15)
    X = np.random.rand(n_train_reg, 1) * np.pi * 2.
    y = np.sin(X)
    y += np.random.randn(n_train_reg, 1) * 0.2
    return StandardScaler().fit_transform(X), y.flatten()

def get_linear_classification_1f():
    # generate classification dataset with 1 informative feature
    np.random.seed(15)
    mean = [0, 0]
    cov = [[1, 0], [0, 10]]
    X = np.zeros((n_train_clf, 2))
    X[:n_train_clf//2] = np.random.multivariate_normal(mean, cov, n_train_clf//2)
    mean = [5, 0]
    X[n_train_clf//2:] = np.random.multivariate_normal(mean, cov, n_train_clf//2)
    y = np.zeros(n_train_clf, dtype=int)
    y[n_train_clf//2:] = 1
    rndidx = np.random.permutation(len(y))
    return StandardScaler().fit_transform(X[rndidx]), y[rndidx]

def get_linear_classification_2f():
    # generate classification dataset with 2 informative features
    np.random.seed(15)
    mean = [0, 4]
    cov = np.array([[1, 8], [8, 10]])
    cov = np.dot(cov, cov.T)/10
    X = np.zeros((n_train_clf, 2))
    X[:n_train_clf//2] = np.random.multivariate_normal(mean, cov, n_train_clf//2)
    mean = [4, 0]
    X[n_train_clf//2:] = np.random.multivariate_normal(mean, cov, n_train_clf//2)
    y = np.zeros(n_train_clf, dtype=int)
    y[n_train_clf//2:] = 1
    rndidx = np.random.permutation(len(y))
    return StandardScaler().fit_transform(X[rndidx]), y[rndidx]

def get_nonlinear_classification():
    # generate non-linear classification dataset
    X, y = make_moons(n_samples=n_train_clf, noise=0.3, random_state=1)
    return StandardScaler().fit_transform(X), y

## Datasets

Here you can have a look at the 3 regression and 3 classification datasets on which we'll compare the different models. The regression dataset only has one input feature, while the classification dataset has two and the target (i.e., class label) is indicated by the color of the dots.

**Questions:**
- Why are the first two regression and classification datasets linear and the last ones non-linear?

In [ ]:
# generate & plot regression datasets
X_reg_1, y_reg_1 = get_linear_regression()
X_reg_2, y_reg_2 = get_linear_outlier()
X_reg_3, y_reg_3 = get_nonlinear_regression()
plot_regression(X_reg_1, y_reg_1)
plot_regression(X_reg_2, y_reg_2)
plot_regression(X_reg_3, y_reg_3)

In [ ]:
# generate & plot classification datasets
X_clf_1, y_clf_1 = get_linear_classification_1f()
X_clf_2, y_clf_2 = get_linear_classification_2f()
X_clf_3, y_clf_3 = get_nonlinear_classification()
plot_classification(X_clf_1, y_clf_1)
plot_classification(X_clf_2, y_clf_2)
plot_classification(X_clf_3, y_clf_3)

## Decision Trees

You can test the Decision Trees here on different datasets (by changing the number at the end of the dataset variable, e.g., `X_reg_1` -> `X_reg_2`) and experiment with their hyperparameter settings (in the comments you'll find a description of the different hyperparameters and which values you can test for them).

**Questions:**
- On the 3rd regression dataset with `max_depth=2`, why do you get exactly 4 plateaus in the prediction?
- On the 3rd regression dataset, what happens if you leave `min_samples_leaf` at 10 and then increase `max_depth` step by step from 2 to 10 or even higher values? How do you explain this behavior and what would you need to do to get a tree that fits the data in a more fine granular way?
- Compare the prediction of the decision tree classifier on the 2nd dataset (which is basically a rotation of the 1st dataset, i.e., still a simple linear classification problem!) to the prediction made by the logistic regression model on this dataset: What do you observe and why?

In [ ]:
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree

### Classification

In [ ]:
# Decision Tree for classification:
# max_depth (>= 1): depth of the tree (i.e. how many decisions are made before the final prediction)
# min_samples_leaf (>= 1): how many training points are in one prediction bucket
X, y = X_clf_1, y_clf_1
model = DecisionTreeClassifier(max_depth=2, min_samples_leaf=10)
model.fit(X, y)
plot_classification(X, y, model)

Let's look at the fitted Decision Tree:

In [ ]:
plot_tree(model, feature_names=['feature 1', 'feature 2'], class_names=['class 0 (red)', 'class 1 (blue)'], filled=True, impurity=False);

### Regression

In [ ]:
# Decision Tree for regression:
# max_depth (>= 1): depth of the tree (i.e. how many decisions are made before the final prediction)
# min_samples_leaf (>= 1): how many training points are in one prediction bucket
X, y = X_reg_1, y_reg_1
model = DecisionTreeRegressor(max_depth=2, min_samples_leaf=10)
model.fit(X, y)
plot_regression(X, y, model)

Let's look at the fitted Decision Tree:

In [ ]:
plot_tree(model, feature_names=['feature 1'], filled=True, impurity=False);